In [21]:
import serial
from tensorflow.keras.models import load_model
import timeit
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
import copy

%matplotlib inline

In [22]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [23]:
ser = serial.Serial("/dev/cu.usbserial-AD01TCNO", baudrate = 115200, timeout = 1)
time.sleep(1)
ser.flushInput()
ser.flushOutput()
time.sleep(1)
model = load_model("../models/model_full_6_points_more_data.h5")
scaler = joblib.load('../scalers/scaler.gz')

In [24]:
def get_serial():
    data = ser.readline()
    data = str(data).replace("b'", "").replace("'", "")
    data = data.rstrip("\\r\\n")
    data = data.split(",")

    try:
        if len(data) == 6:
            return [int(data[0]), float(data[1]), float(data[2]), int(data[3]), int(data[4]), int(data[5])]
        else:
            return data
    except ValueError:
        return data

In [25]:
def plot(values, pred, pred_val, time):
    plt.figure(2)
    plt.clf()
    plt.title("Training...")
    plt.xlabel("Episode")
    plt.ylabel("Duration")
    plt.plot(values)
    plt.pause(0.001)
    print(pred[0][0], pred_val[0][0], "Time: ", time)
    if is_ipython: display.clear_output(wait = True)

In [26]:
def is_spike_up(threshold_up, threshold_down, x_window):
    spike = [None, None]

    for i in range(len(x_window) - 1):
        if x_window[i + 1] > x_window[i] * threshold_up:
            spike[0] = i
            
    if spike[0] != None:
        for i in range(spike[0], len(x_window) - 1):
            if x_window[i + 1] < x_window[i] * threshold_down:
                spike[1] = i
            
    return spike
            
def is_spike_down(threshold_up, threshold_down, x_window):
    spike = [None, None]

    for i in range(len(x_window) - 1):
        if x_window[i + 1] < x_window[i] * threshold_down:
            spike[0] = i
            
    if spike[0] != None:
        for i in range(spike[0], len(x_window) - 1):
            if x_window[i + 1] > x_window[i] * threshold_up:
                spike[1] = i
            
    return spike
        
def get_is_spike(x, x_window):
    threshold_up = 50
    threshold_down = 0.02
    is_spike = False
    spike = [None, None]
    
    spike_up = is_spike_up(threshold_up, threshold_down, x_window)
    spike_down = is_spike_down(threshold_up, threshold_down, x_window)
    
    if spike_up[0] != None and spike_up[1] != None:
        is_spike = True
        spike = spike_up
    elif spike_down[0] != None and spike_down[1] != None:
        is_spike = True
        spike = spike_down
        
    return (is_spike, spike)


def get_filterd_data(x):
    x = np.array(x)
    for i in range(len(x)):
        if i > 1 and i < len(x) - 2:
            for j in range(len(x[i])):
                x_window = x[i - 2 : i + 3][:, j] # need to be odd
                is_spike, spike_window_indexes = get_is_spike(x[i][j], x_window)
                if is_spike:

                    x[i][j] = np.median(x_window)
    return x

In [32]:
def predict(input_data):
    print(input_data.reshape(6, -1).shape)
    print(input_data)
    prediction = model.predict([input_data[0], input_data[1], input_data[2], input_data[3], input_data[4], input_data[5]])
    return (prediction > 0.6, prediction)

In [33]:
last_data = []
is_flying = False
count = 0
while True:

    serial_data = get_serial()
    if isinstance(serial_data[0], int) and isinstance(serial_data[1], float) and isinstance(serial_data[2], float):
        last_data.append(serial_data)

    if len(last_data) >= 100:
        start_prediction = timeit.default_timer()
        flex_data = np.array(last_data).reshape(100, 6)
        flex_data = get_filterd_data(flex_data)
        flex_data_scaled = scaler.transform(flex_data)

        pred, pred_val = predict(flex_data_scaled)
        stop_prediction = timeit.default_timer()
        
        pred_time = stop_prediction - start_prediction

        if is_ipython: display.clear_output(wait = True)
        print(pred[0][0], pred_val[0][0], "Time: ", pred_time)
        
        if pred:
            last_data = []
            start_data_collection = timeit.default_timer()
        else:
            last_data = last_data[-85:]

(6, 100)
[[-0.26293103  0.48001894  0.51597983  0.48291504  0.50577541  0.60909984]
 [-0.26293103  0.47990754  0.5159241   0.48284552  0.50569553  0.60905074]
 [-0.26293103  0.47996324  0.51584051  0.482776    0.50574345  0.60908347]
 [-0.26293103  0.47996324  0.51589624  0.48284552  0.50574345  0.60909984]
 [-0.26293103  0.47996324  0.51581264  0.482776    0.50574345  0.60913257]
 [-0.26293103  0.47996324  0.51581264  0.482776    0.50574345  0.6091162 ]
 [-0.26293103  0.47996324  0.51578478  0.48272386  0.50574345  0.609018  ]
 [-0.26293103  0.47996324  0.51575692  0.48272386  0.50574345  0.60913257]
 [-0.26293103  0.47999109  0.51575692  0.48272386  0.50574345  0.60919804]
 [-0.26293103  0.47999109  0.51578478  0.48275862  0.50575943  0.60914894]
 [-0.26293103  0.48004678  0.51581264  0.482776    0.50579138  0.60914894]
 [-0.26293103  0.48004678  0.51586837  0.48282814  0.50579138  0.60924714]
 [-0.26293103  0.48004678  0.5159241   0.48288028  0.50579138  0.60934534]
 [-0.26293103  0

ValueError: Error when checking input: expected input_1 to have 3 dimensions, but got array with shape (6, 1)